In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, depolarizing_error,
                              amplitude_damping_error, ReadoutError)
from tqdm.notebook import tqdm

In [ ]:
# #config

# MAX_QUBITS = 4
# MAX_GATES = 50 #instead of 30
# VECTOR_SIZE = 16
# N_SAMPLES = 10000 #5000

# NOISE_LEVELS = {
#     "low": 0.05,
#     "moderate": 0.1,
#     "high": 0.15
# }

In [ ]:
# #helper fns

# def get_noise_model(p_err):
#     noise_model = NoiseModel()

#     # 1. Depolarizing
#     error_depol_1q = depolarizing_error(p_err, 1)

#     # 2. Amplitude Damping (makes the qubit drift towards 0)
#     error_damp_1q = amplitude_damping_error(p_err)

#     #composition
#     full_error_1q = error_depol_1q.compose(error_damp_1q)

#     # adding errors to 1qbit gates
#     noise_model.add_all_qubit_quantum_error(full_error_1q, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'id', 'sx', 'x'])

#     # adding errors to 2qubit gates
#     error_depol_2q = depolarizing_error(p_err * 2, 2) #doubling depolarizing strength
#     error_damp_2q = error_damp_1q.tensor(error_damp_1q) #assuming both qubits dampen independently

#     full_error_2q = error_depol_2q.compose(error_damp_2q)
#     noise_model.add_all_qubit_quantum_error(full_error_2q, ['cx'])

#     # 3. Readout Error
#     p_ro = p_err
#     readout_error = ReadoutError([[1 - p_ro, p_ro], [p_ro, 1 - p_ro]])
#     noise_model.add_all_qubit_readout_error(readout_error)

#     return noise_model

# # from raw measurements to prob vector
# def counts_to_vector(counts, n_qubits):
#     total_shots = sum(counts.values())
#     prob_vector = np.zeros(VECTOR_SIZE)
#     for bitstring, count in counts.items():
#         index = int(bitstring, 2)
#         prob_vector[index] = count / total_shots
#     return prob_vector

In [ ]:
# data_rows = []

# pbar = tqdm(total=N_SAMPLES)

# while len(data_rows) < N_SAMPLES:
#     try: #fixed the error of non transpiled circuits

#         n_qubits = np.random.randint(2, MAX_QUBITS+1)
#         depth = np.random.randint(2,9)
#         qc = random_circuit(n_qubits, depth, measure=True)

#         if sum(dict(qc.count_ops()).values()) > MAX_GATES:
#             continue

#         row = {
#             "n_qubits": n_qubits,
#             "n_gates": sum(dict(qc.count_ops()).values()),
#             "depth": depth
#         }

#         # PART ONE: THE IDEAL SIMULATION
#         ideal_sim = AerSimulator()
#         # forcing basis gates to prevent the crash that happened
#         t_qc_ideal = transpile(qc, ideal_sim, basis_gates=['u1', 'u2', 'u3', 'cx', 'id', 'rz', 'sx', 'x'])
#         job = ideal_sim.run(t_qc_ideal, shots=10000)
#         vec_ideal = counts_to_vector(job.result().get_counts(), n_qubits)

#         for i in range(VECTOR_SIZE):
#             row[f"ideal_{i}"] = vec_ideal[i]

#         # PART TWO: THE NOISY SIMULATION
#         for level_name, p_err in NOISE_LEVELS.items():
#             nm = get_noise_model(p_err)
#             noisy_sim = AerSimulator(noise_model=nm)
#             t_qc_noisy = transpile(qc, noisy_sim, basis_gates=['u1', 'u2', 'u3', 'cx', 'id', 'rz', 'sx', 'x'])
#             job = noisy_sim.run(t_qc_noisy, shots=10000)
#             vec_noisy = counts_to_vector(job.result().get_counts(), n_qubits)

#             for i in range(VECTOR_SIZE):
#                 row[f"{level_name}_{i}"] = vec_noisy[i]

#         data_rows.append(row)
#         pbar.update(1)

#     except Exception as e:
#         continue

# pbar.close()

# df = pd.DataFrame(data_rows)
# df.to_csv("quantum_dataset.csv", index=False)
# print("All done :)")

# try:
#     from google.colab import files
#     files.download('quantum_dataset.csv')
# except ImportError:
#     pass

  0%|          | 0/10000 [00:00<?, ?it/s]

All done :)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

References:
1. https://quantum.cloud.ibm.com/docs/en/api/qiskit/circuit_random
2. https://qiskit.github.io/qiskit-aer/tutorials/3_building_noise_models.html

In [ ]:
MAX_QUBITS = 4
MAX_GATES = 50
VECTOR_SIZE = 16

N_CIRCUITS = 10000
NOISE_MAP = {
    "low": 0,       #  0 for Low noise
    "moderate": 1,  #  1 for Moderate noise
    "high": 2       #  2 for High noise
}

NOISE_PROBS = {
    "low": 0.05,
    "moderate": 0.1,
    "high": 0.15
}

In [ ]:
def get_noise_model(p_err):
    noise_model = NoiseModel()

    # 1. Depolarizing
    error_depol_1q = depolarizing_error(p_err, 1)

    # 2. Amplitude Damping (makes the qubit drift towards 0)
    error_damp_1q = amplitude_damping_error(p_err)

    #composition
    full_error_1q = error_depol_1q.compose(error_damp_1q)

    # adding errors to 1qbit gates
    noise_model.add_all_qubit_quantum_error(full_error_1q, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'id', 'sx', 'x'])

    # adding errors to 2qubit gates
    error_depol_2q = depolarizing_error(p_err * 2, 2) #doubling depolarizing strength
    error_damp_2q = error_damp_1q.tensor(error_damp_1q) #assuming both qubits dampen independently

    full_error_2q = error_depol_2q.compose(error_damp_2q)
    noise_model.add_all_qubit_quantum_error(full_error_2q, ['cx'])

    # 3. Readout Error
    p_ro = p_err
    readout_error = ReadoutError([[1 - p_ro, p_ro], [p_ro, 1 - p_ro]])
    noise_model.add_all_qubit_readout_error(readout_error)

    return noise_model

# from raw measurements to prob vector
def counts_to_vector(counts, n_qubits):
    total_shots = sum(counts.values())
    prob_vector = np.zeros(VECTOR_SIZE)
    for bitstring, count in counts.items():
        index = int(bitstring, 2)
        prob_vector[index] = count / total_shots
    return prob_vector

In [ ]:
data_rows = []

# Progress bar tracks the number of UNIQUE circuits generated
pbar = tqdm(total=N_CIRCUITS)
circuits_done = 0

while circuits_done < N_CIRCUITS:
    try:
        n_qubits = np.random.randint(2, MAX_QUBITS+1)
        depth = np.random.randint(2,9)
        qc = random_circuit(n_qubits, depth, measure=True)

        if sum(dict(qc.count_ops()).values()) > MAX_GATES:
            continue

        # Basic metadata shared by all 3 versions of this circuit
        base_info = {
            "n_qubits": n_qubits,
            "n_gates": sum(dict(qc.count_ops()).values()),
            "depth": depth
        }

        # PART ONE: THE IDEAL SIMULATION (Calculated once per circuit)
        ideal_sim = AerSimulator()
        t_qc_ideal = transpile(qc, ideal_sim, basis_gates=['u1', 'u2', 'u3', 'cx', 'id', 'rz', 'sx', 'x'])
        job = ideal_sim.run(t_qc_ideal, shots=10000)
        vec_ideal = counts_to_vector(job.result().get_counts(), n_qubits)

        # PART TWO: THE NOISY SIMULATIONS (Loop through the 3 levels)
        for level_name, p_err in NOISE_PROBS.items():

            # Create a FRESH row for this specific noise level
            row = base_info.copy()

            # 1. Add the Noise Level Feature (0, 1, or 2)
            row["noise_level"] = NOISE_MAP[level_name]

            # 2. Run Noisy Simulation
            nm = get_noise_model(p_err)
            noisy_sim = AerSimulator(noise_model=nm)
            t_qc_noisy = transpile(qc, noisy_sim, basis_gates=['u1', 'u2', 'u3', 'cx', 'id', 'rz', 'sx', 'x'])
            job = noisy_sim.run(t_qc_noisy, shots=10000)
            vec_noisy = counts_to_vector(job.result().get_counts(), n_qubits)

            # 3. Save Noisy Vector (The Input)
            for i in range(VECTOR_SIZE):
                row[f"noisy_{i}"] = vec_noisy[i]

            # 4. Save Ideal Vector (The Target) - Same for all 3 rows
            for i in range(VECTOR_SIZE):
                row[f"ideal_{i}"] = vec_ideal[i]

            # Add this single sample to the dataset
            data_rows.append(row)

        circuits_done += 1
        pbar.update(1)

    except Exception as e:
        continue

pbar.close()

In [3]:
df = pd.DataFrame(data_rows)
#shuffling dataset so Low/Med/High aren't always next to each other
df = df.sample(frac=1).reset_index(drop=True)

print(f"Total Circuits Generated: {circuits_done}")
print(f"Total Dataset Samples: {len(df)}") #should be ~30,000

df.to_csv("quantum_dataset_long.csv", index=False)
print("All done :)")

try:
    from google.colab import files
    files.download('quantum_dataset_long.csv')
except ImportError:
    pass

  0%|          | 0/10000 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(
/usr/local/lib/python3.12/dist-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(


Total Circuits Generated: 10000
Total Dataset Samples: 30000
All done :)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>